>这篇文章主要通过几个简单的例子来演示神经网络

# 0.目标
从一组输入的样本中找到规律，并生成一个程序（函数），接受新的输入，预测结果。
# 1.输入输出
众所周知，在计算机内部数据是以二进制按顺序存放在存储空间的，**一组有序的数字** 在数学中叫做**向量**，所以对于神经网络而言，可以认为输入都是一个向量，同理，输出也可以当作向量处理。

>在编程语言里，通常用一维数组表示向量

绝大多数时候，我们所用的数据不止一组，**同时向量组可以用矩阵表示**，所以我们把输入输出都当作矩阵。
> 这里涉及一个输入模式输出对的集合（训练数据）。每个输入向量都有一个对应的期望输出向量、或者称作是目标向量。
# 2.一个简单的例子
考虑输入为$X=\left(
\begin{array}{ccc}
 0 & 0 & 1 \\
 0 & 1 & 1 \\
 1 & 0 & 1 \\
 1 & 1 & 1 \\
\end{array}
\right)$，输出为$Y=\left(
\begin{array}{c}
 0 \\
 1 \\
 1 \\
 1 \\
\end{array}
\right)$的一组数据。


In [1]:
import numpy as np
X=np.array([[0,0,1],
           [0,1,1],
           [1,0,1],
           [1,1,1]])
Y=np.array([[0,1,1,1]]).transpose()



我们先动用真正的人工智能（就是我们自己）找下规律：

*  显然，第三列对输出没有影响，然后一、二列和输出呈现出跟**或门** 类似的关系。


下面我们来让计算机自动的从输入输出中学习这种规律（所谓的机器学习）。
现在我们要将一个$4*3$的矩阵映射到$4*1$的矩阵中去，最简单直观的做法的用一个$3*1$的矩阵$W$与$X$相乘。
$$Y=XW$$
那么，现在的目标就是求这个$W$（称作权值）.

>注：在这$W$的大小与输入的向量组的数量（$X$的列数）无关，所以不管有多少组数据，对于这个模型而言$W$还是$3*1$的



首先，我们用随机数初始化$W$:
$$W=\left(
\begin{array}{c}
 \text{RandomReal} \\
 \text{RandomReal} \\
 \text{RandomReal} \\
\end{array}
\right)=\left(
\begin{array}{c}
 0.639875 \\
 0.0906838 \\
 0.14675 \\
\end{array}
\right)$$


In [2]:
W=np.random.rand(3,1)


然后计算$\hat{Y}=X W=\left(
\begin{array}{c}
 0.14675 \\
 0.237434 \\
 0.786625 \\
 0.877309 \\
\end{array}
\right)$,

In [3]:
Y_=X.dot(W)



这里有一个问题，因为$W$元素取值在$R$ 上，所以$\hat Y $有可能超过$1$，这对于这个模型而言显然是错误的。

所以我们要找一个函数$f(\hat Y)$,使得其输出的范围在$0 \sim 1$之间。
在这里我选用$\sigma$ 函数，因为$\sigma^{'}=\sigma(1-\sigma) $,它的导数较为简单。

$\sigma $的图像如下：
![enter image description here](https://leanote.com/api/file/getImage?fileId=58e051c3ab64413a33003e06)
> 选用$\sigma$的原因不止这一个，后面会介绍具体原因和其他类似的函数



所以现在
$$\hat{Y}=\sigma( XW)=\left(
\begin{array}{c}
 0.536622 \\
 0.559081 \\
 0.687106 \\
 0.706264 \\
\end{array}
\right)$$


In [4]:
def sigmod(X):
    return 1/(1+np.exp(-X))
print(sigmod(np.array([-10,-1,0,1,10])))
Y_=sigmod(X.dot(W))
Y_

[4.53978687e-05 2.68941421e-01 5.00000000e-01 7.31058579e-01
 9.99954602e-01]


array([[0.72581509],
       [0.86279314],
       [0.82377759],
       [0.91738579]])


然后我们计算一下当前权值$W$的误差：$\hat e=Y- \hat Y=\left(
\begin{array}{c}
 -0.536622 \\
 0.440919 \\
 0.312894 \\
 0.293736 \\
\end{array}
\right)$,

In [5]:
e_=Y-Y_
e_

array([[-0.72581509],
       [ 0.13720686],
       [ 0.17622241],
       [ 0.08261421]])



可以看到误差还是相当大的，我们根据这些误差的大小正负更新一下$W$：

$$\Delta W=\hat e * \sigma^ {' }(\hat Y)$$

$$W=W+X^T \Delta W$$
> 上面两条式子也称作**Delta规则**，具体推导过程在以后的文章里有介绍，这一篇文章只是引入，不深究细节


In [6]:
def Dsigmod(X):
    return sigmod(X)*(1-sigmod(X))
d_W=e_*Dsigmod(Y_)
print("old W:",W)
W=W+X.transpose().dot(d_W)
print("new W:",W)

old W: [[0.56866112]
 [0.86519267]
 [0.97349257]]
new W: [[0.62286541]
 [0.91067876]
 [0.89682311]]


最后，我们重复上面的步骤，一直更新$W$，使它趋近于我们所要求.(这个模型较为简单,迭代1000次就行)


In [7]:
print("old W:",W)

for i in range(1000):
    Y_=sigmod(X.dot(W))
    e_=Y-Y_
    d_W=e_*Dsigmod(Y_)
    W=W+X.transpose().dot(d_W)
print("new W:",W)

old W: [[0.62286541]
 [0.91067876]
 [0.89682311]]
new W: [[ 8.43929978]
 [ 8.43957152]
 [-3.87322625]]


关于新输入的预测，只要拿到计算好的$W$,对于任意一组新的输入$X_i$,可以求得新的输出$\hat Y_i=\sigma( X_i W)$.

In [8]:
Xi=np.array([[0,1,0]])
Yi_=sigmod(Xi.dot(W))
Yi_

array([[0.9997839]])


# 3.神经网络

上面都在讲矩阵的运算，那么跟神经网络有什么关系？

先来看一组例子$X=(0,0,1) \to Y=(0)$,我们用圆点代表元素项，把输入的每一项与输出连接起来，


![title](https://leanote.com/api/file/getImage?fileId=58e0567eab64413779003f4b)

然后把$W$的每一项按顺序加到每一条边当做权值，注意到在上面的模型里还有一个$\sigma$函数，将它当做一个虚拟的元素项（节点），最后画出来就是一个赋权图。

![title](https://leanote.com/api/file/getImage?fileId=58e05b41ab64413779004036)

对$\sigma$ 这个节点而言，它接收从$X$传进来的三个值，然后加权求和后，通过$\sigma$函数生成Y的预测值。这时$\sigma$节点就好像大脑的一个神经元一样，接收别的神经元的电信号，再产生新的电信号给别的神经元。
当多个神经元相互连接，就组成一个**神经网络**。
> 或者叫做**感知机**



# 4.学习能力

对于输入为$X=\left(
\begin{array}{ccc}
 0 & 0 & 1 \\
 0 & 1 & 1 \\
 1 & 0 & 1 \\
 1 & 1 & 1 \\
\end{array}
\right)$，输出为$Y=\left(
\begin{array}{c}
 0 \\
 1 \\
 1 \\
 1 \\
\end{array}
\right)$的一组数据。我们可以用上面的模型学习其中的规律，并预测，假如我们把输出改成$Y=\left(
\begin{array}{c}
 0 \\
 1 \\
 1 \\
 0 \\
\end{array}
\right)$，也就是 一二两列的关系改成**异或**，这个模型就不能成功模拟了。
> 可以自行用代码验证，或者利用$\hat{Y}=\sigma( X W)$这个式子，用反证法证明不存在一个W，使得对上述的输入输出成立（或者近似成立）。
> **它的本质是因为简单的感知机不能解决异或问题**

下面是验证的效果：



In [9]:
X=np.array([[0,0,1],
           [0,1,1],
           [1,0,1],
           [1,1,1]])
Y=np.array([[0,1,1,0]]).transpose()
W=np.random.rand(3,1)

print("old W:",W)

for i in range(1000):
    Y_=sigmod(X.dot(W))
    e_=Y-Y_
    d_W=e_*Dsigmod(Y_)
    W=W+X.transpose().dot(d_W)
print("new W:",W)

old W: [[0.13264637]
 [0.47475026]
 [0.83667792]]
new W: [[-4.57966998e-16]
 [-4.57966998e-16]
 [ 5.82867088e-16]]




最终的结果会陷入一个最优解中，但不是我们所需要的。

可以这样理解，因为当前模型的学习能力太弱，无法学习出高级的函数。

> 如果一个这么简单模型能够学习出所有的函数，那我们还要研究什么。


假如我们把模型变得复杂（就是增加神经元），那就能学习出更高级的函数了。


# 5.增加神经元
对于这个神经网络：
![title](https://leanote.com/api/file/getImage?fileId=58e05b41ab64413779004036)

我们把$\sigma$神经元copy一下，增加复杂度

![enter image description here](https://leanote.com/api/file/getImage?fileId=58e0c70bab64413779005685)

注意到第三行的$\sigma$,它把第二行的输出当做输入，同样加权求和，输出Y的预测值。

下面写出它数学形式
> 有了一个模型的数学形式，你就可以用任意的编程语言编写出来了

输入为$X=\left(
\begin{array}{ccc}
 0 & 0 & 1 \\
 0 & 1 & 1 \\
 1 & 0 & 1 \\
 1 & 1 & 1 \\
\end{array}
\right)$，输出为$Y=\left(
\begin{array}{c}
 0 \\
 1 \\
 1 \\
 0 \\
\end{array}
\right)$.

In [10]:
X=np.array([[0,0,1],
           [0,1,1],
           [1,0,1],
           [1,1,1]])
Y=np.array([[0,1,1,0]]).transpose()



令$\sigma_i=\sigma(XW_i )\ \  \ \ i=1,2,3,4$ 
与上面的同理，对于任意的$W_i$而言，都是大小为3*1的列向量，那么$W=(W_1，W_2，W_3,W_4)$为$3 * 4$的矩阵，同样用随机数初始化。

则$L=(\sigma_1,\sigma_2,\sigma_3,\sigma_4)$是W和X的乘积，也就是4*4的矩阵。

我们得到第一条式子
$$L=\sigma(XW)$$

In [11]:
W=np.random.rand(3,4)
L=sigmod(X.dot(W))



对于上面模型中的$\sigma$ 节点，我们也可以类似的写出

$$\hat Y =\sigma (L W_2 )$$

其中，$W_2$是指第二行到第三行的权值，与$W=(W_1，W_2，W_3,W_4)$中的$W_2$意义不同。（找不到别的符号表示了，就重复了= =）。这里的$W_2$ 应该是一个4*1的列向量。

In [12]:
W2=np.random.rand(4,1)
Y_=sigmod(L.dot(W2))



然后计算下误差$\hat e=Y- \hat Y$，更新权值$W_2$.


$$\Delta W_2=\hat e * \sigma^ {' }(\hat Y)$$

$$W_2=W_2+L^T \Delta W_2$$
注意到，$W_2=W_2+L^T \Delta W_2$ 是以$L$的转置与增量相乘，而不与第一个模型一样的用$X^T$,因为**应该用这个神经元的输入与增量相乘**.


In [13]:
e_=Y-Y_
dW2 = e_* Dsigmod(Y_)
W2 = W2 +L.transpose().dot(dW2)


>或者可以认为$X$的大小不对，没法相乘？？？

算出L的误差（可以直接复用上面算好的$\Delta W_2$）：$\hat e_L = \Delta W_2 W_2^T$

In [14]:
eL_ = dW2.dot(W2.transpose())

更新权值$W$也应该用对应的输入输出算出的增量

$$\Delta W=\hat e_L * \sigma^ {' }(L)$$
$$W=W+X^T \Delta W$$

In [15]:
dW= eL_* Dsigmod(L)
W= W+X.transpose().dot(dW)

In [23]:
X=np.array([[0,0,1],
           [0,1,1],
           [1,0,1],
           [1,1,1]])
Y=np.array([[0,1,1,0]]).transpose()
W=np.random.rand(3,4)
W2=np.random.rand(4,1)
for i in range(10000):
    L=sigmod(X.dot(W))
    Y_=sigmod(L.dot(W2))
    e_=Y-Y_
    dW2 = e_* Dsigmod(Y_)
    eL_ = dW2.dot(W2.transpose())
    dW= eL_* Dsigmod(L)
    W2 = W2 +L.transpose().dot(dW2)
    W= W+X.transpose().dot(dW)
L=sigmod(X.dot(W))
Y_=sigmod(L.dot(W2))
Y_

array([[0.02033941],
       [0.97208095],
       [0.97153743],
       [0.0243601 ]])

# 6.整理
$ X$ 是我们的模型的输入,$Y$  是模型的输出,

在第一个模型里,我们从输入到输出经过:
$$ X  \xrightarrow{\text{W} } Y$$

在第二个模型里,我们从输入到输出经过:
$$ X  \xrightarrow{W_1 } L \xrightarrow{W _{2}}  Y$$
## 6.1 分层

不管是从X直接到Y,还是要经过L到Y,可以注意到L和Y的值都与上一个变量(X或L)的值相关.这样的层我们称作全连接层.

In [4]:
import numpy as np
class Input():
    def __init__(self,X):
        self.X=X
        self.outsize=X.shape[1]
    def forward(self):
        return self.X
    
class Dense():
    def __init__(self,above,outsize,acfn,Dacfn):
        self.outsize=outsize
        self.acfn=acfn
        self.X=above.X
        self.W=np.random.rand(above.outsize,outsize)
    def forward(self):

        return self.acfn(self.X.dot(self.W))
    def forback(self,Y):
        Y_=self.forwrad(self.X)
        e_=Y-Y_
        dW = e_* Dacfn(Y_)
        self.W = self.W+X.transpose().dot(dW)

In [5]:
X=np.array([[0,0,1],
           [0,1,1],
           [1,0,1],
           [1,1,1]])
Y=np.array([[0,1,1,1]]).transpose()

In [7]:
def sigmod(X):
    return 1/(1+np.exp(-X))
def Dsigmod(X):
    return sigmod(X)*(1-sigmod(X))

input=Input(X)
out=Dense(input,1,sigmod,Dsigmod)

In [8]:
out.forward()

array([[0.65573155],
       [0.69979996],
       [0.78618794],
       [0.81818729]])

In [9]:
class Model():
    list=[]
    def add (layer):
        list.append(layer)
    def train(count):
        for i in range (count):
            Y=list[-1].forward()
            for j in range(list.count,0,-1):
                Y=list[j].forback(Y)